In [ ]:
import torch
from models import MVCNN
from torchvision import transforms
import os
from PIL import Image
import re
import random

In [55]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

In [56]:
svcnn = MVCNN.SVCNN(name="VGG11_SVCNN")
svcnn_weights = torch.load(
    "trained-models/SVCNN/model-00030.pth", weights_only=True, map_location=device
)
svcnn.load_state_dict(svcnn_weights, strict=False)
svcnn.eval()
svcnn.to(device)

mvcnn = MVCNN.MVCNN(name="VGG11_MVCNN", model=svcnn)
mvcnn_weights = torch.load(
    "trained-models/MVCNN/model-00030.pth", weights_only=True, map_location=device
)
mvcnn.load_state_dict(mvcnn_weights, strict=False)
mvcnn.eval()
mvcnn.to(device)

MVCNN(
  (net_1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  

In [57]:
transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225])
])

In [58]:
def init_csv():
    with open("svcnn-grads.csv", "w") as file:
        file.write("model,model_id,imp1,imp2,imp3,imp4,imp5,imp6,imp7,imp8,imp9,imp10,imp11,imp12\n")
    print("File Initialised")

In [59]:
def save_to_csv(grads: dict, model: str, model_id: int):
    with open ('svcnn-grads.csv', 'a') as file:
        file.write(f"{model},{model_id},")
        for i in range(0, 12):
            file.write(f"{grads[i]},")
        file.write("\n")
    print("File Updated")

In [60]:
def run_svcnn(svcnn, images_dir):
    outputs = []
    images = os.listdir(images_dir)
    images = sorted(
        images, key=lambda x: int(re.search(r'_(\d+)\.png', x)[1])
    )
    # images = [f"{images_dir}/{image}" for image in images]
    images = [os.path.join(images_dir, image) for image in images]
    svcnn = svcnn.to(device)
    svcnn.eval()
    # print(images)
    for _ in range(12):
        image = Image.open(images[_])
        image = image.convert('RGB')
        image = transforms(image)
        image = image.to(device)
        image = image.unsqueeze(0)
        image = image.to(device)
        with torch.no_grad():
            output = svcnn(image)
        outputs.append(output)
    pooled_features, _ = torch.max(torch.stack(outputs), dim=0, keepdim=False)
    pooled_features = pooled_features.view(1, -1)
    pooled_features = pooled_features.to(device)
    # print(pooled_features.shape)
    return pooled_features

In [61]:
path ='./ModelNet40-12View/'
objects = [obj for obj in os.listdir(path) if not obj.startswith('.')] 
object_paths = [os.path.join(path, i, 'test') for i in objects]

model_paths = []
for obj_path in object_paths:
    if not os.path.isdir(obj_path):
        continue
    models_in_folder = [
        os.path.join(obj_path, fname)
        for fname in os.listdir(obj_path)
        if not fname.startswith('.')  # Exclude .DS_Store or other hidden files
    ]
    model_paths.extend(models_in_folder)

In [62]:
def rem_dup(arr):
    done = []
    for i in arr:
        if i not in done:
            done.append(i)
    return done

In [63]:
class_names=['airplane','bed','bench','bookshelf','bottle','bowl','car','chair',
                'cone','cup','curtain','door','flower_pot','glass_box',
                'guitar','keyboard','lamp','laptop','mantel', 'person','piano',
                'plant','radio','range_hood','sink','stairs',
                'stool','tent','toilet','tv_stand','vase','wardrobe','xbox']

classes = {}
for i in range(len(class_names)):
    classes[class_names[i]] = i
    print(f"Class Name: {class_names[i]}, index: {i}")

Class Name: airplane, index: 0
Class Name: bed, index: 1
Class Name: bench, index: 2
Class Name: bookshelf, index: 3
Class Name: bottle, index: 4
Class Name: bowl, index: 5
Class Name: car, index: 6
Class Name: chair, index: 7
Class Name: cone, index: 8
Class Name: cup, index: 9
Class Name: curtain, index: 10
Class Name: door, index: 11
Class Name: flower_pot, index: 12
Class Name: glass_box, index: 13
Class Name: guitar, index: 14
Class Name: keyboard, index: 15
Class Name: lamp, index: 16
Class Name: laptop, index: 17
Class Name: mantel, index: 18
Class Name: person, index: 19
Class Name: piano, index: 20
Class Name: plant, index: 21
Class Name: radio, index: 22
Class Name: range_hood, index: 23
Class Name: sink, index: 24
Class Name: stairs, index: 25
Class Name: stool, index: 26
Class Name: tent, index: 27
Class Name: toilet, index: 28
Class Name: tv_stand, index: 29
Class Name: vase, index: 30
Class Name: wardrobe, index: 31
Class Name: xbox, index: 32


In [64]:
count = 0
random.shuffle(model_paths)
tests = len(model_paths)
correct = 0
print(f"{'Count':<8}{'Model':<20}{'Model Class':<20}{'Actual Class':<15}{'Predicted Class':<20}{'Accuracy':<15}")
print("--------------------------------------------------------"*2)
for model_path in model_paths:
    count+=1
    pooled_features = run_svcnn(svcnn.net_1, model_path)
    with torch.no_grad():
        mvcnn = mvcnn.to(device)
        mvcnn.eval()
        pooled_features = pooled_features.to(device)
        final_out = torch.argmax(mvcnn.net_2(pooled_features), dim=1)
    model = model_path.split('/')[2]
    model_class = model_path.split('/')[4]
    print(f"{count:<8}{model:<20}{model_class:<20}{classes[model]:<15}{final_out.item():<20}{'Correct' if int(classes[model]) == int(final_out.item()) else 'Wrong':<15}")
    if int(classes[model]) == int(final_out.item()):
        correct+=1
    if count > tests:
        break
    
print(f"Accuracy: {correct}/{count} = {correct/count*100}%")

Count   Model               Model Class         Actual Class   Predicted Class     Accuracy       
----------------------------------------------------------------------------------------------------------------
1       vase                vase_0508           30             26                  Wrong          
2       range_hood          range_hood_0135     23             18                  Wrong          
3       glass_box           glass_box_0234      13             13                  Correct        
4       keyboard            keyboard_0149       15             3                   Wrong          
5       glass_box           glass_box_0173      13             13                  Correct        
6       bowl                bowl_0079           5              5                   Correct        
7       cone                cone_0181           8              30                  Wrong          
8       plant               plant_0254          21             21                  Correct     